<a href="https://colab.research.google.com/github/greyhound101/Orbuculum/blob/main/download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [1]:
%%writefile download_abide_preproc.py
def collect_and_download(derivative, pipeline, strategy, out_dir, less_than, greater_than, site, sex, diagnosis):
    """
    Function to collect and download images from the ABIDE preprocessed
    directory on FCP-INDI's S3 bucket
    Parameters
    ----------
    derivative : string
        derivative or measure of interest
    pipeline : string
        pipeline used to process data of interest
    strategy : string
        noise removal strategy used to process data of interest
    out_dir : string
        filepath to a local directory to save files to
    less_than : float
        upper age (years) threshold for participants of interest
    greater_than : float
        lower age (years) threshold for participants of interest
    site : string
        acquisition site of interest
    sex : string
        'M' or 'F' to indicate whether to download male or female data
    diagnosis : string
        'asd', 'tdc', or 'both' corresponding to the diagnosis of the
        participants for whom data should be downloaded
    Returns
    -------
    None
        this function does not return a value; it downloads data from
        S3 to a local directory
    :param derivative: 
    :param pipeline: 
    :param strategy: 
    :param out_dir: 
    :param less_than: 
    :param greater_than: 
    :param site: 
    :param sex:
    :param diagnosis:
    :return: 
    """

    # Import packages
    import os
    import urllib.request as request

    # Init variables
    mean_fd_thresh = 0.2
    s3_prefix = 'https://s3.amazonaws.com/fcp-indi/data/Projects/'\
                'ABIDE_Initiative'
    s3_pheno_path = '/'.join([s3_prefix, 'Phenotypic_V1_0b_preprocessed1.csv'])

    # Format input arguments to be lower case, if not already
    derivative = derivative.lower()
    pipeline = pipeline.lower()
    strategy = strategy.lower()

    # Check derivative for extension
    if 'roi' in derivative:
        extension = '.1D'
    else:
        extension = '.nii.gz'

    # If output path doesn't exist, create it
    if not os.path.exists(out_dir):
        print('Could not find {0}, creating now...'.format(out_dir))
        os.makedirs(out_dir)

    # Load the phenotype file from S3
    s3_pheno_file = request.urlopen(s3_pheno_path)
    pheno_list = s3_pheno_file.readlines()
    print(pheno_list[0])

    # Get header indices
    header = pheno_list[0].decode().split(',')
    try:
        site_idx = header.index('SITE_ID')
        file_idx = header.index('FILE_ID')
        age_idx = header.index('AGE_AT_SCAN')
        sex_idx = header.index('SEX')
        dx_idx = header.index('DX_GROUP')
        mean_fd_idx = header.index('func_mean_fd')
    except Exception as exc:
        err_msg = 'Unable to extract header information from the pheno file: {0}\nHeader should have pheno info:' \
                  ' {1}\nError: {2}'.format(s3_pheno_path, str(header), exc)
        raise Exception(err_msg)

    # Go through pheno file and build download paths
    print('Collecting images of interest...')
    s3_paths = []
    for pheno_row in pheno_list[1:]:

        # Comma separate the row
        cs_row = pheno_row.decode().split(',')

        try:
            # See if it was preprocessed
            row_file_id = cs_row[file_idx]
            # Read in participant info
            row_site = cs_row[site_idx]
            row_age = float(cs_row[age_idx])
            row_sex = cs_row[sex_idx]
            row_dx = cs_row[dx_idx]
            row_mean_fd = float(cs_row[mean_fd_idx])
        except Exception as e:
            err_msg = 'Error extracting info from phenotypic file, skipping...'
            print(err_msg)
            continue

        # If the filename isn't specified, skip
        if row_file_id == 'no_filename':
            continue
        # If mean fd is too large, skip
        if row_mean_fd >= mean_fd_thresh:
            continue

        # Test phenotypic criteria (three if's looks cleaner than one long if)
        # Test sex
        if (sex == 'M' and row_sex != '1') or (sex == 'F' and row_sex != '2'):
            continue

        if (diagnosis == 'asd' and row_dx != '1') or (diagnosis == 'tdc' and row_dx != '2'):
            continue

        # Test site
        if site is not None and site.lower() != row_site.lower():
            continue
        # Test age range
        if greater_than < row_age < less_than:
            filename = row_file_id + '_' + derivative + extension
            s3_path = '/'.join([s3_prefix, 'Outputs', pipeline, strategy, derivative, filename])
            print('Adding {0} to download queue...'.format(s3_path))
            s3_paths.append(s3_path)
        else:
            continue

    # And download the items
    total_num_files = len(s3_paths)
    for path_idx, s3_path in enumerate(s3_paths):
        rel_path = s3_path.lstrip(s3_prefix)
        download_file = os.path.join(out_dir, rel_path)
        download_dir = os.path.dirname(download_file)
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
        try:
            if not os.path.exists(download_file):
                print('Retrieving: {0}'.format(download_file))
                request.urlretrieve(s3_path, download_file)
                print('{0:3f}% percent complete'.format(100*(float(path_idx+1)/total_num_files)))
            else:
                print('File {0} already exists, skipping...'.format(download_file))
        except Exception as exc:
            print('There was a problem downloading {0}.\n Check input arguments and try again.'.format(s3_path))

    # Print all done
    print('Done!')


# Make module executable
if __name__ == '__main__':

    # Import packages
    import argparse
    import os
    import sys

    # Init argument parser
    parser = argparse.ArgumentParser(description=__doc__)

    # Required arguments
    parser.add_argument('-a', '--asd', required=False, default=False, action='store_true',
                        help='Only download data for participants with ASD.'
                             ' Specifying neither or both -a and -c will download data from all participants.')
    parser.add_argument('-c', '--tdc', required=False, default=False, action='store_true',
                        help='Only download data for participants who are typically developing controls.'
                             ' Specifying neither or both -a and -c will download data from all participants.')
    parser.add_argument('-d', '--derivative', nargs=1, required=True, type=str,
                        help='Derivative of interest (e.g. \'reho\')')
    parser.add_argument('-p', '--pipeline', nargs=1, required=True, type=str,
                        help='Pipeline used to preprocess the data (e.g. \'cpac\')')
    parser.add_argument('-s', '--strategy', nargs=1, required=True, type=str,
                        help='Noise-removal strategy used during preprocessing (e.g. \'nofilt_noglobal\'')
    parser.add_argument('-o', '--out_dir', nargs=1, required=True, type=str,
                        help='Path to local folder to download files to')

    # Optional arguments
    parser.add_argument('-lt', '--less_than', nargs=1, required=False,
                        type=float, help='Upper age threshold (in years) of participants to download (e.g. for '
                                         'subjects 30 or younger, \'-lt 31\')')
    parser.add_argument('-gt', '--greater_than', nargs=1, required=False,
                        type=int, help='Lower age threshold (in years) of participants to download (e.g. for '
                                       'subjects 31 or older, \'-gt 30\')')
    parser.add_argument('-t', '--site', nargs=1, required=False, type=str,
                        help='Site of interest to download from (e.g. \'Caltech\'')
    parser.add_argument('-x', '--sex', nargs=1, required=False, type=str,
                        help='Participant sex of interest to download only (e.g. \'M\' or \'F\')')

    # Parse and gather arguments
    args = parser.parse_args()

    # Init variables
    desired_derivative = args.derivative[0].lower()
    desired_pipeline = args.pipeline[0].lower()
    desired_strategy = args.strategy[0].lower()
    download_data_dir = os.path.abspath(args.out_dir[0])

    # Try and init optional arguments

    # for diagnosis if both ASD and TDC flags are set to true or false, we download both
    desired_diagnosis = ''
    if args.tdc == args.asd:
        desired_diagnosis = 'both'
        print('Downloading data for ASD and TDC participants')
    elif args.tdc:
        desired_diagnosis = 'tdc'
        print('Downloading data for TDC participants')
    elif args.asd:
        desired_diagnosis = 'asd'
        print('Downloading data for ASD participants')

    try:
        desired_age_max = args.less_than[0]
        print('Using upper age threshold of {0:d}...'.format(desired_age_max))
    except TypeError:
        desired_age_max = 200.0
        print('No upper age threshold specified')

    try:
        desired_age_min = args.greater_than[0]
        print('Using lower age threshold of {0:d}...'.format(desired_age_min))
    except TypeError:
        desired_age_min = -1.0
        print('No lower age threshold specified')

    try:
        desired_site = args.site[0]
    except TypeError:
        desired_site = None
        print('No site specified, using all sites...')

    try:
        desired_sex = args.sex[0].upper()
        if desired_sex == 'M':
            print('Downloading only male subjects...')
        elif desired_sex == 'F':
            print('Downloading only female subjects...')
        else:
            print('Please specify \'M\' or \'F\' for sex and try again')
            sys.exit()
    except TypeError:
        desired_sex = None
        print('No sex specified, using all sexes...')

    # Call the collect and download routine
    collect_and_download(desired_derivative, desired_pipeline, desired_strategy, download_data_dir, desired_age_max,
                         desired_age_min, desired_site, desired_sex, desired_diagnosis)

Writing download_abide_preproc.py


In [9]:
!python3 download_abide_preproc.py -d 'rois_ho' -p 'cpac' -s 'filt_noglobal' -t "UM_1" -o'/content/gdrive/MyDrive/federated'

No upper age threshold specified
No lower age threshold specified
No sex specified, using all sexes...
b',Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,COMORBIDITY,CURRENT_MED_STATUS,MEDICATION_NAME,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,VINELAND_EXPRESSIVE_V_SCALED,VINELAND_WRITTEN_V_SCALED,VINELAND_COMMUNICATION_STANDARD,VINELAND_PERSONAL_V_SCALED,VINELAND_DOMESTIC_V_SCALED,VINELAND_COMMUNITY_V_SCALED,VINELAND_DAILYLVNG_STANDARD,VINELAND_INTERPERSONAL_V_SCALED,VINELAND_PLAY_V

In [10]:
!python3 download_abide_preproc.py -d 'rois_ho' -p 'cpac' -s 'filt_noglobal' -t "NYU" -o'/content/gdrive/MyDrive/federated'

No upper age threshold specified
No lower age threshold specified
No sex specified, using all sexes...
b',Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,COMORBIDITY,CURRENT_MED_STATUS,MEDICATION_NAME,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,VINELAND_EXPRESSIVE_V_SCALED,VINELAND_WRITTEN_V_SCALED,VINELAND_COMMUNICATION_STANDARD,VINELAND_PERSONAL_V_SCALED,VINELAND_DOMESTIC_V_SCALED,VINELAND_COMMUNITY_V_SCALED,VINELAND_DAILYLVNG_STANDARD,VINELAND_INTERPERSONAL_V_SCALED,VINELAND_PLAY_V

In [11]:
!python3 download_abide_preproc.py -d 'rois_ho' -p 'cpac' -s 'filt_noglobal' -t "USM" -o'/content/gdrive/MyDrive/federated'

No upper age threshold specified
No lower age threshold specified
No sex specified, using all sexes...
b',Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,COMORBIDITY,CURRENT_MED_STATUS,MEDICATION_NAME,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,VINELAND_EXPRESSIVE_V_SCALED,VINELAND_WRITTEN_V_SCALED,VINELAND_COMMUNICATION_STANDARD,VINELAND_PERSONAL_V_SCALED,VINELAND_DOMESTIC_V_SCALED,VINELAND_COMMUNITY_V_SCALED,VINELAND_DAILYLVNG_STANDARD,VINELAND_INTERPERSONAL_V_SCALED,VINELAND_PLAY_V

In [12]:
!python3 download_abide_preproc.py -d 'rois_ho' -p 'cpac' -s 'filt_noglobal' -t "UCLA_1" -o'/content/gdrive/MyDrive/federated'

No upper age threshold specified
No lower age threshold specified
No sex specified, using all sexes...
b',Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,COMORBIDITY,CURRENT_MED_STATUS,MEDICATION_NAME,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,VINELAND_EXPRESSIVE_V_SCALED,VINELAND_WRITTEN_V_SCALED,VINELAND_COMMUNICATION_STANDARD,VINELAND_PERSONAL_V_SCALED,VINELAND_DOMESTIC_V_SCALED,VINELAND_COMMUNITY_V_SCALED,VINELAND_DAILYLVNG_STANDARD,VINELAND_INTERPERSONAL_V_SCALED,VINELAND_PLAY_V

In [ ]:
/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0051066_rois_ho.1D

In [17]:
rm '/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050302_rois_ho.1D'

In [18]:
rm '/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050320_rois_ho.1D'

In [21]:
rm '/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050273_rois_ho.1D'

In [23]:
rm '/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050280_rois_ho.1D'

In [24]:
rm '/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050313_rois_ho.1D'

rm: cannot remove '/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050313_rois_ho.1D': No such file or directory


In [25]:
paths=['/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050302_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050320_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050273_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050280_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050313_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050367_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050354_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050352_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050353_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050362_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050301_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050316_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050324_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050282_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050336_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050343_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050330_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UM_1_0050298_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050532_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050497_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050520_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050505_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050510_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050526_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050466_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050491_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050492_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050507_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050435_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050528_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050449_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050467_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050490_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050498_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050463_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050501_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/USM_0050437_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0051110_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0050961_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0051123_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0051155_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0050958_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0051114_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0051058_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/NYU_0051118_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051281_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051236_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051220_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051266_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051260_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051212_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051227_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051271_rois_ho.1D'
,'/content/gdrive/MyDrive/federated/Outputs/cpac/filt_noglobal/rois_ho/UCLA_1_0051272_rois_ho.1D']

In [26]:
import os
for path in paths:
  try:
    os.remove(path)
  except:
    pass